<a href="https://colab.research.google.com/github/ZDuysen/UMadina-DZhanerke/blob/main/HE_BIOm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

MFA

In [40]:
import random

# Simulate user database with usernames and passwords
user_database = {
    "madina": "password123",
    "zhanerke": "securepass456",
    "madinazhanerkee": "letmein789"
}

# Generate a random 6-digit OTP
def generate_otp():
    return str(random.randint(100000, 999999))

# Verify the username and password
def verify_password(username, password):
    if username in user_database and user_database[username] == password:
        return True
    else:
        return False

# Verify the OTP
def verify_otp(otp, expected_otp):
    return otp == expected_otp

# Main authentication process
def authenticate():
    username = input("Enter your username: ")
    password = input("Enter your password: ")

    if verify_password(username, password):
        expected_otp = generate_otp()
        print(f"OTP sent to your mobile device: {expected_otp}")
        otp = input("Enter the OTP: ")

        if verify_otp(otp, expected_otp):
            print("Authentication successful!")
        else:
            print("Invalid OTP. Authentication failed.")
    else:
        print("Invalid username or password. Authentication failed.")

if __name__ == "__main__":
    authenticate()


Enter your username: madina
Enter your password: password123
OTP sent to your mobile device: 652923
Enter the OTP: 652923
Authentication successful!


In [41]:
!pip install tenseal
!pip install base64
import tenseal as ts
import base64

ERROR: Could not find a version that satisfies the requirement base64 (from versions: none)
ERROR: No matching distribution found for base64


In [42]:
context = ts.context(ts.SCHEME_TYPE.CKKS, poly_modulus_degree = 8192, coeff_mod_bit_sizes = [60, 40, 40, 60])
context.generate_galois_keys()
context.global_scale = 2**40

In [43]:
def write_data(file_name, data):
    if type(data) == bytes:
        #bytes to base64
        data = base64.b64encode(data)

    with open(file_name, 'wb') as f:
        f.write(data)

def read_data(file_name):
    with open(file_name, "rb") as f:
        data = f.read()
    #base64 to bytes
    return base64.b64decode(data)

In [44]:
secret_context = context.serialize(save_secret_key = True)
write_data("secret.txt", secret_context)

context.make_context_public() #drop the secret_key from the context
public_context = context.serialize()
write_data("public.txt", public_context)

In [45]:
!pip install deepface
from deepface import DeepFace

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 kB 12.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.2/80.2 kB 10.2 MB/s eta 0:00:00
  Created wheel for fire: filename=fire-0.5.0-py2.py3-none-any.whl size=116934 sha256=754efe4d382d63b295235c382dad3b279190a536737235b593f61ce5d0d1f120
  Stored in directory: /root/.cache/pip/wheels/90/d4/f7/9404e5db0116bd4d43e5666eaa3e70ab53723e1e3ea40c9a95
Successfully built fire
Directory  /root /.deepface created
Directory  /root /.deepface/weights created


In [46]:


img1_path = "img1.jpg"
img2_path = "img5.jpg"

img1_embedding = DeepFace.represent(img1_path, model_name = 'Facenet')[0]["embedding"]
img5_embedding = DeepFace.represent(img2_path, model_name = 'Facenet')[0]["embedding"]


facenet_weights.h5 will be downloaded...


Downloading...
From: https://github.com/serengil/deepface_models/releases/download/v1.0/facenet_weights.h5
To: /root/.deepface/weights/facenet_weights.h5
100%|██████████| 92.2M/92.2M [00:01<00:00, 50.1MB/s]


In [47]:
context = ts.context_from(read_data("secret.txt"))

enc_v1 = ts.ckks_vector(context, img1_embedding)
enc_v5 = ts.ckks_vector(context, img5_embedding)

enc_v1_proto = enc_v1.serialize()
enc_v5_proto = enc_v5.serialize()

write_data("enc_v1.txt", enc_v1_proto)
write_data("enc_v5.txt", enc_v5_proto)


In [48]:
#cloud system will have the public key
context = ts.context_from(read_data("public.txt"))

#restore the embedding of person 1
enc_v1_proto = read_data("enc_v1.txt")
enc_v1 = ts.lazy_ckks_vector_from(enc_v1_proto)
enc_v1.link_context(context)

#restore the embedding of person 2
enc_v5_proto = read_data("enc_v5.txt")
enc_v5 = ts.lazy_ckks_vector_from(enc_v5_proto)
enc_v5.link_context(context)

#euclidean distance
euclidean_squared = enc_v1 - enc_v5
euclidean_squared = euclidean_squared.dot(euclidean_squared)

#store the homomorphic encrypted squared euclidean distance
write_data("euclidean_squared.txt", euclidean_squared.serialize())


In [49]:
try:
    euclidean_squared.decrypt()
except Exception as err:
    print("Exception: ", str(err))


Exception:  the current context of the tensor doesn't hold a secret_key, please provide one as argument


In [50]:
#client has the secret key
context = ts.context_from(read_data("secret.txt"))

#load euclidean squared value
euclidean_squared_proto = read_data("euclidean_squared.txt")
euclidean_squared = ts.lazy_ckks_vector_from(euclidean_squared_proto)
euclidean_squared.link_context(context)

#decrypt it
euclidean_squared_plain = euclidean_squared.decrypt()[0]
euclidean_squared_plain

56.05416793572937

In [51]:
if euclidean_squared_plain < 100:
    print("they are same person")
else:
    print("they are different persons")


they are same person


In [52]:
from deepface.commons import distance as dst

distance = dst.findEuclideanDistance(img1_embedding, img5_embedding)
squared_distance = distance * distance
squared_distance

56.05415899196307